In [1]:
%config IPCompleter.greedy=True

In [2]:
import pandas as pd
import numpy as np
np.random.seed(0)

In [3]:

checkin = pd.read_csv("./Datasets/dataset_TIST2015/smallCheckin.csv", header=None, sep="\t")
cities = pd.read_csv("./Datasets/dataset_TIST2015/dataset_TIST2015_Cities.txt", header=None, sep="\t")
pois = pd.read_csv("./Datasets/dataset_TIST2015/dataset_TIST2015_POIs.txt", header=None, sep="\t")

checkin.columns = ['user_id','venue_id','utc_time','offset']
pois.columns = ['venue_id','latitude','longitude','category','country_code']
cities.columns = ['city','latitude','longitude','country_code','country_name','city_type']

df1 = checkin.merge(pois, on="venue_id")


In [21]:
v = set()
size = 60
def chunk_preprocess(df,field='user_id'):
    global v,size
    df.columns= ['user_id','venue_id','utc_time','offset']
    d = df[df[field].isin(v)]
    return d

In [22]:
import random
random.seed(0)
size =1000

In [29]:
checkin_chunk = pd.read_csv("./Datasets/dataset_TIST2015/dataset_TIST2015_Checkins.txt", header=None,sep="\t",chunksize = 150000)
checkins_list = []
for df in checkin_chunk:
    if(len(v) >= size):
        break
    df.columns= ['user_id','venue_id','utc_time','offset']
    if(len(df['user_id'].unique()) < size-len(v)):
        v = v | len(df['userid'].unique())
    else:
        v = v | set(random.sample(set(df['user_id'].unique()), size-len(v)))
                              
print('len v', len(v))
    
for ind, i in enumerate(checkin_chunk):
    df  = chunk_preprocess(i)
    if not ind%50: print('ind ', ind)
    if df is not None: 
        checkins_list.append(df)
        #print('subframe len',len(df.index))
        #print('No. of users: ', len(df['user_id'].unique()))
        # print(len(checkins_list))
    else:
        print("000000000000")
        break
if len(checkins_list) >= 1: checkindf = pd.concat(checkins_list)
else: print("000000000000")
print('final len', len(checkindf.index))
print('No. of users: ', len(checkindf['user_id'].unique()))

len v 1000
ind  0
ind  50
ind  100
ind  150
ind  200
final len 181089
No. of users:  1000


In [30]:
checkindf.to_csv("./Datasets/dataset_TIST2015/smalldata_unproc.csv", index=False)

In [31]:
c = pd.read_csv("./Datasets/dataset_TIST2015/smalldata_unproc.csv")

In [32]:

c = c.drop('offset', axis=1)


In [33]:
c.head()

,user_id,venue_id,utc_time
0,133809,4d563f3e9e508cfa7f210b9b,Wed Apr 04 23:51:35 +0000 2012
1,143349,4c3a3bd7dfb0e21ebfdbb1a8,Wed Apr 04 23:52:24 +0000 2012
2,35454,4b60e62cf964a52068002ae3,Wed Apr 04 23:52:27 +0000 2012
3,26272,4f73eae8bb3dcf63c7ef8d89,Wed Apr 04 23:52:39 +0000 2012
4,80886,4b635c54f964a52082742ae3,Wed Apr 04 23:52:46 +0000 2012


In [34]:
len(c.index)

181089

In [35]:
c['utc_time'] = pd.to_datetime(c['utc_time'])


In [36]:
c['utc_time'] = c['utc_time'] - c['utc_time'].min()


In [37]:
c['utc_time'] = c['utc_time'].dt.seconds


In [38]:
c.to_csv("./Datasets/dataset_TIST2015/smalldata_unproc.csv", index=False)
c.head()

,user_id,venue_id,utc_time
0,133809,4d563f3e9e508cfa7f210b9b,0
1,143349,4c3a3bd7dfb0e21ebfdbb1a8,49
2,35454,4b60e62cf964a52068002ae3,52
3,26272,4f73eae8bb3dcf63c7ef8d89,64
4,80886,4b635c54f964a52082742ae3,71


In [40]:
############# Start from here ##################
c = pd.read_csv("./Datasets/dataset_TIST2015/smalldata_unproc.csv")
c.head()

,user_id,venue_id,utc_time
0,133809,4d563f3e9e508cfa7f210b9b,0
1,143349,4c3a3bd7dfb0e21ebfdbb1a8,49
2,35454,4b60e62cf964a52068002ae3,52
3,26272,4f73eae8bb3dcf63c7ef8d89,64
4,80886,4b635c54f964a52082742ae3,71


In [41]:
df= c.merge(pois,on="venue_id")


In [42]:
len(df.index)

181089

In [45]:
df.head()

,user_id,venue_id,utc_time,latitude,longitude,category,country_code
0,133809,4d563f3e9e508cfa7f210b9b,0,51.062714,-113.952671,Home (private),CA
1,133809,4d563f3e9e508cfa7f210b9b,4395,51.062714,-113.952671,Home (private),CA
2,133809,4d563f3e9e508cfa7f210b9b,1443,51.062714,-113.952671,Home (private),CA
3,133809,4d563f3e9e508cfa7f210b9b,70401,51.062714,-113.952671,Home (private),CA
4,133809,4d563f3e9e508cfa7f210b9b,20050,51.062714,-113.952671,Home (private),CA


In [46]:
#df = df.drop(['latitude','longitude'], axis=1)
len(np.unique(df['category']))


420

In [47]:
d = {k:v for v,k in enumerate(list(set(df['category'])))}
print('No. of cats: ', len(d.keys()))
df['category_id'] = df['category'].map(d)

('No. of cats: ', 420)


In [48]:
df.head()

,user_id,venue_id,utc_time,latitude,longitude,category,country_code,category_id
0,133809,4d563f3e9e508cfa7f210b9b,0,51.062714,-113.952671,Home (private),CA,7
1,133809,4d563f3e9e508cfa7f210b9b,4395,51.062714,-113.952671,Home (private),CA,7
2,133809,4d563f3e9e508cfa7f210b9b,1443,51.062714,-113.952671,Home (private),CA,7
3,133809,4d563f3e9e508cfa7f210b9b,70401,51.062714,-113.952671,Home (private),CA,7
4,133809,4d563f3e9e508cfa7f210b9b,20050,51.062714,-113.952671,Home (private),CA,7


In [49]:
d = {k:v for v,k in enumerate(list(set(df['country_code'])))}
print('No. of countries: ', len(d.keys()))
df['country_code_id'] = df['country_code'].map(d)

('No. of countries: ', 71)


In [50]:
df.head()

,user_id,venue_id,utc_time,latitude,longitude,category,country_code,category_id,country_code_id
0,133809,4d563f3e9e508cfa7f210b9b,0,51.062714,-113.952671,Home (private),CA,7,38
1,133809,4d563f3e9e508cfa7f210b9b,4395,51.062714,-113.952671,Home (private),CA,7,38
2,133809,4d563f3e9e508cfa7f210b9b,1443,51.062714,-113.952671,Home (private),CA,7,38
3,133809,4d563f3e9e508cfa7f210b9b,70401,51.062714,-113.952671,Home (private),CA,7,38
4,133809,4d563f3e9e508cfa7f210b9b,20050,51.062714,-113.952671,Home (private),CA,7,38


In [51]:
df.head()

,user_id,venue_id,utc_time,latitude,longitude,category,country_code,category_id,country_code_id
0,133809,4d563f3e9e508cfa7f210b9b,0,51.062714,-113.952671,Home (private),CA,7,38
1,133809,4d563f3e9e508cfa7f210b9b,4395,51.062714,-113.952671,Home (private),CA,7,38
2,133809,4d563f3e9e508cfa7f210b9b,1443,51.062714,-113.952671,Home (private),CA,7,38
3,133809,4d563f3e9e508cfa7f210b9b,70401,51.062714,-113.952671,Home (private),CA,7,38
4,133809,4d563f3e9e508cfa7f210b9b,20050,51.062714,-113.952671,Home (private),CA,7,38


In [52]:
len(set(df['user_id']))

1000

In [53]:
df.sort_values(by=['user_id','utc_time'],inplace=True)
df.to_csv("./Datasets/dataset_TIST2015/smalldata.csv", index=False, sep='\t')

In [4]:
df = pd.read_csv("./Datasets/dataset_TIST2015/smalldata.csv", sep='\t')
venue_city = dict()

In [5]:
len(set(df['venue_id']))

65030

In [6]:
df.head()

,user_id,venue_id,utc_time,latitude,longitude,category,country_code,category_id,country_code_id
0,260,4aa9386ef964a5208b5220e3,22,40.745837,-73.987993,American Restaurant,US,100,60
1,260,4b021b27f964a520bf4722e3,28,40.760108,-74.044416,Restaurant,US,335,60
2,260,4bfaa8a35efe2d7fb3ea6d34,58,40.867217,-73.896747,Bus Line,US,364,60
3,260,4a0e0f85f964a520bf751fe3,90,40.745878,-73.988142,Hotel,US,377,60
4,260,4bc9f113cc8cd13a7c7fbccf,98,40.757713,-74.043751,Fried Chicken Joint,US,164,60


In [7]:
[1,2] == [1,2]

True

In [8]:
venue_city

{}

In [10]:
import pickle
f = open("./Datasets/dataset_TIST2015/citylist_latlong.pkl", "rb")
venue_city = pickle.load(f)
f.close()

In [18]:
len(venue_city)

50977

In [13]:
a = 0
def makeCity(l):
    global a
    if (l['latitude'], l['longitude']) in venue_city.keys():
        return venue_city[(l['latitude'], l['longitude'])]
    else:
        a += 1
        return np.nan

df['city_name'] = df.apply(makeCity, axis=1)
df.head()


,user_id,venue_id,utc_time,latitude,longitude,category,country_code,category_id,country_code_id,city_name
0,260,4aa9386ef964a5208b5220e3,22,40.745837,-73.987993,American Restaurant,US,100,60,New York
1,260,4b021b27f964a520bf4722e3,28,40.760108,-74.044416,Restaurant,US,335,60,Union City
2,260,4bfaa8a35efe2d7fb3ea6d34,58,40.867217,-73.896747,Bus Line,US,364,60,New York
3,260,4a0e0f85f964a520bf751fe3,90,40.745878,-73.988142,Hotel,US,377,60,New York
4,260,4bc9f113cc8cd13a7c7fbccf,98,40.757713,-74.043751,Fried Chicken Joint,US,164,60,Union City


In [13]:
!pip3 install tqdm

  Using cached https://files.pythonhosted.org/packages/47/55/fd9170ba08a1a64a18a7f8a18f088037316f2a41be04d2fe6ece5a653e8f/tqdm-4.43.0-py2.py3-none-any.whl


In [23]:
df.head(10)

,user_id,venue_id,utc_time,latitude,longitude,category,country_code,category_id,country_code_id,city_name
0,260,4aa9386ef964a5208b5220e3,22,40.745837,-73.987993,American Restaurant,US,100,60,New York
1,260,4b021b27f964a520bf4722e3,28,40.760108,-74.044416,Restaurant,US,335,60,Union City
2,260,4bfaa8a35efe2d7fb3ea6d34,58,40.867217,-73.896747,Bus Line,US,364,60,New York
3,260,4a0e0f85f964a520bf751fe3,90,40.745878,-73.988142,Hotel,US,377,60,New York
4,260,4bc9f113cc8cd13a7c7fbccf,98,40.757713,-74.043751,Fried Chicken Joint,US,164,60,Union City
5,260,4c4ec7fc9932e21e36b249ce,105,40.865904,-73.894830,Home (private),US,7,60,New York
6,260,4cd3f20d9828b60cc31ad884,142,40.825079,-73.920135,City,US,290,60,New York
7,260,4f8213e6e4b042dc225bf065,143,40.745025,-74.006368,Park,US,243,60,New York
8,260,42f15a80f964a520bc261fe3,173,40.752120,-73.999760,Event Space,US,362,60,New York
9,260,4da4d091fa8c7394fc86d456,198,40.832561,-73.915054,Residential Building (Apartment / Condo),US,265,60,New York


In [24]:
df['city_name'].to_list().count(np.nan)

39216

In [25]:
a

39216

In [27]:
df.to_csv("./Datasets/dataset_TIST2015/smalldata.csv", index=False, sep='\t', encoding='utf-8')

In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv("./Datasets/dataset_TIST2015/smalldata_part.csv", sep = "\t")
df.head()

/home/rishi/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,user_id,venue_id,utc_time,latitude,longitude,category,country_code,category_id,country_code_id,city_name
0,260,4aa9386ef964a5208b5220e3,22.0,40.745837,-73.987993,American Restaurant,US,100.0,60.0,New York
1,260,4b021b27f964a520bf4722e3,28.0,40.760108,-74.044416,Restaurant,US,335.0,60.0,Union City
2,260,4bfaa8a35efe2d7fb3ea6d34,58.0,40.867217,-73.896747,Bus Line,US,364.0,60.0,New York
3,260,4a0e0f85f964a520bf751fe3,90.0,40.745878,-73.988142,Hotel,US,377.0,60.0,New York
4,260,4bc9f113cc8cd13a7c7fbccf,98.0,40.757713,-74.043751,Fried Chicken Joint,US,164.0,60.0,Union City


In [3]:
df.dtypes

user_id             object
venue_id            object
utc_time           float64
latitude           float64
longitude          float64
category            object
country_code        object
category_id        float64
country_code_id    float64
city_name           object
dtype: object

In [4]:
df.tail()

,user_id,venue_id,utc_time,latitude,longitude,category,country_code,category_id,country_code_id,city_name
181084,263463,43ca6968f964a520af2d1fe3,80502.0,34.057388,-118.237980,Plaza,US,148.0,60.0,Los Angeles
181085,263463,4b27d842f964a520098b24e3,82705.0,34.020645,-118.399512,Coffee Shop,US,22.0,60.0,NaN
181086,263463,4b27d842f964a520098b24e3,82800.0,34.020645,-118.399512,Coffee Shop,US,22.0,60.0,NaN
181087,263463,4de41d6a18385df2b04c8b3d,83058.0,34.007673,-118.412474,Ramen / Noodle House,US,153.0,60.0,NaN
181088,263463,4de41d6a18385df2b04c8b3d,84692.0,34.007673,-118.412474,Ramen / Noodle House,US,153.0,60.0,NaN
